https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py

In [1]:
%matplotlib inline
import tensorflow as tf
from utils.celeb import *
from utils.dataset import *
from utils.reporting import *

from IPython.display import display
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import IPython.display as ipyd
plt.style.use('ggplot')

In [2]:
#data_dir = "/home/data/images/"
#log_dir = "/home/logs/"

data_dir = "../data/MsCelebV1-Faces-Cropped.Samples/"
log_dir = "./test/logs"

In [3]:
n_entities = 4
n_images = 10
seed = 42

In [4]:
face_cascade = cv2.CascadeClassifier('./utils/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./utils/haarcascade_eye.xml')

In [5]:
df = get_dataframe(data_dir, n_entities, n_images, rnd_seed=seed)
print('I have a df')
all_images, all_labels = get_all_images(df, data_dir)
print('I have images and labels')
all_images = np.ravel(all_images).reshape(all_images.shape[0], all_images.shape[1] * all_images.shape[2] * all_images.shape[3])

I have a df
I have images and labels


In [6]:
split = [0.8, 0.0, 0.2]
ds = Dataset(all_images, all_labels, split=split, one_hot=True, rnd_seed=seed)

n_samples = ds.X.shape[0]
n_features = ds.X.shape[1]
n_classes = ds.Y.shape[1]

Dataset init


### Create placeholders

In [7]:
# Input placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, n_features], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, n_classes], name='y-input')

with tf.name_scope('input_reshape'):
    #x_4d = tf.reshape(x, [-1, 50, 50, 3])
    x_4d = tf.reshape(x, [-1, 100, 100, 3])
    tf.summary.image('input', x_4d, 10)

### Create Variables for Weights and Variables

In [8]:
# We can't initialize these variables to 0 - the network will get stuck.

def weight_variable_xavier(name, shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.get_variable(
                            name=name,
                            shape=shape,
                            initializer=tf.contrib.layers.xavier_initializer_conv2d())
    return initial

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, strides=[1,1,1,1]):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

def max_pool_2x2(x, ksize, strides):
    return tf.nn.max_pool(x, ksize=ksize,
                        strides=strides, padding='SAME')

In [9]:
#X must be 4D
#def convolutions(X, n_input_chs, n_filters, filter_sizes, filter_strides, activation=tf.nn.relu):
def convolutions(X, n_filters, filter_sizes, filter_strides, activation=tf.nn.relu):
    current_in = X
    n_input_chs = X.get_shape().as_list()[3]
    Ws = []
    shapes = []

    # Build the encoder
    for layer_i, n_output_chs in enumerate(n_filters):
        with tf.variable_scope('convolution/{}'.format(layer_i)):
            shapes.append(current_in.get_shape().as_list())
            W = weight_variable_xavier(
                                name = "W{}".format(layer_i),
                                shape = [filter_sizes[layer_i], 
                                 filter_sizes[layer_i], 
                                 n_input_chs, n_output_chs])
            
            h = conv2d(current_in, W, filter_strides)
            h = activation(h)
            Ws.append(W)
            current_in = h
            n_input_chs = n_output_chs
            print("h.shape:", h.get_shape().as_list())
    shapes.append(current_in.get_shape().as_list())
    print("shapes:",shapes)
    return h, Ws

### Create Summaries for all var to include min/max/mean/std_dev

In [10]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

### Fully connected layer

In [11]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    if (len(input_tensor.get_shape().as_list())==4):
        input_shape = input_tensor.get_shape().as_list()
        input_tensor = tf.reshape(input_tensor, [-1, input_shape[1]*input_shape[2]*input_shape[3]])
    
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations, preactivate

In [12]:
def drop_layer(h, dropout):
    with tf.name_scope('dropout'):
        #keep_prob = tf.placeholder(tf.float32)
        #tf.summary.scalar('dropout_keep_probability', keep_prob)
        tf.summary.scalar('dropout_keep_probability', dropout)
        dropped = tf.nn.dropout(h, dropout)
    return dropped

# Configuration

In [13]:
#data_dir = "/home/data/images/"
#log_dir = "/home/logs/"
learning_rate = 0.003
epochs = 100
dropout = 0.8
FC_only = False

#Conv parameters
n_filters = [32, 32, 8]  #filter output sizes
filter_sizes = [4, 4, 2]  #
filter_strides = [1, 2, 2, 1]
#maxpool parameters
ksize = [1,2,2,1]
k_strides = [1,2,2,1]
#FC parameter
final_layer = [n_classes]

In [14]:
if not FC_only:
    h, Ws = convolutions(x_4d, n_filters, filter_sizes, filter_strides)
    h_shape = h.get_shape().as_list()
    print("h.shape:", h.get_shape().as_list())
    print("h[3]:", h.get_shape().as_list()[3])
    print("Ws:", Ws)

h.shape: [None, 50, 50, 32]
h.shape: [None, 25, 25, 32]
h.shape: [None, 13, 13, 8]
shapes: [[None, 100, 100, 3], [None, 50, 50, 32], [None, 25, 25, 32], [None, 13, 13, 8]]
h.shape: [None, 13, 13, 8]
h[3]: 8
Ws: [<tensorflow.python.ops.variables.Variable object at 0x110fad390>, <tensorflow.python.ops.variables.Variable object at 0x127e86358>, <tensorflow.python.ops.variables.Variable object at 0x127e86940>]


#### Make a FC layer

In [15]:
#hidden1 = nn_layer(h, h, 500, 'layer1')
hidden1, pre_act_1 = nn_layer(h, h_shape[1]*h_shape[2]*h_shape[3], 100, 'layer1')

In [16]:
print(hidden1)

Tensor("layer1/activation:0", shape=(?, 100), dtype=float32)


### Drop layer

In [17]:
# Do not apply softmax activation yet, see below.
#dropped = drop_layer(hidden1)
#y = nn_layer(hidden1, 100, n_classes, 'layer2', act=tf.identity)

dropped = drop_layer(hidden1, dropout)
y, pre_act_2 = nn_layer(dropped, 100, n_classes, 'layer2', act=tf.identity)

### Cross entropy cost function

In [18]:
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

<tf.Tensor 'cross_entropy_1:0' shape=() dtype=string>

In [19]:
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

### Accuracy

In [20]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

### Feed data to feed_dict

In [21]:
def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    #if train or FLAGS.fake_data:
        #xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
        #k = FLAGS.dropout
    if train:
        xs, ys = ds.train.images, ds.train.labels
        k = dropout
    else:    
        xs, ys = ds.test.images, ds.test.labels
        k = 1.0
    #return {x: xs, y_: ys, keep_prob: k}
    return {x: xs, y_: ys}

### Summary writer and Create session

In [22]:
# Merge all the summaries and write them out to /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
merged = tf.summary.merge_all()
#sess = tf.Session()
#sess.run(tf.global_variables_initializer())

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options,
        allow_soft_placement=True, log_device_placement=True))  
sess.run(tf.global_variables_initializer())

train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')

## Train and Test

In [23]:
batch_size = 20

finalRepresentations = []
for i in range(epochs):
    
    if i%10 == 0:
        # Train accuracy report
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
        print('Train accuracy at step %s: %s' % (i, acc))
        finalRepresentations.append(pre_act_2.eval(session=sess, feed_dict=feed_dict(True)))
    else:
        # Train network
        for batch_X, batch_Y in ds.train.next_batch(batch_size=batch_size):
            
            summary, _ = sess.run([merged, train_step], feed_dict={x: batch_X, y_: batch_Y})
            train_writer.add_summary(summary, i)

test_hash = feed_dict(False)
summary, acc, y = sess.run([merged, accuracy, y], feed_dict=feed_dict(False))
test_writer.add_summary(summary, i)
print('Test Accuracy: %s' % (acc))

report(y, ds, n_classes)

train_writer.close()
test_writer.close()
#sess.close()

Train accuracy at step 0: 0.25
Train accuracy at step 10: 0.9375
Train accuracy at step 20: 1.0
Train accuracy at step 30: 1.0
Train accuracy at step 40: 1.0
Train accuracy at step 50: 1.0
Train accuracy at step 60: 1.0
Train accuracy at step 70: 1.0
Train accuracy at step 80: 1.0
Train accuracy at step 90: 1.0
Test Accuracy: 0.5

Train Class Distribution


,0,1,2,3
distribution,9,9,7,7



Test Class Distribution


,0,1,2,3
distribution,1,1,3,3



Train Results


,value
prec ovall,0.688
recall overall,0.667
f1 overall,0.517



Test Results by Class


,0,1,2,3
prec,0.25,0.500,1.000,1.000
recall,1.00,1.000,0.333,0.333
f1,0.40,0.667,0.500,0.500
